In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Installations
!pip install -U tensorflow
!pip install numpy pandas

# Imports
import tensorflow as tf
import numpy as np
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split

# Load all CSV files
deadlift = pd.read_csv('/content/drive/MyDrive/BarbellAI/deadlift_*.csv')
bench = pd.read_csv('/content/drive/MyDrive/BarbellAI/bench_press_*.csv')
squat = pd.read_csv('/content/drive/MyDrive/BarbellAI/squat_*.csv')

data = pd.concat([deadlift, bench, squat])
X = data[['ax', 'ay', 'az', 'gx', 'gy', 'gz']].values
y = data[['rep_phase', 'exercise_id']].values  # Assuming exercise_id column exists

# Normalize
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# Reshape for LSTM
X = X.reshape(-1, 50, 6)  # 50-step sequences
y_rep = y[:, 0]  # rep_phase
y_exercise = y[:, 1]  # exercise_id

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(50, 6)),
    tf.keras.layers.Conv1D(32, 5, activation='relu'),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # [rep_prob, vel_correction, exercise_id]
])

model.compile(optimizer='adam',
              loss=['binary_crossentropy', 'mse', 'sparse_categorical_crossentropy'],
              metrics=['accuracy'])

# Train
model.fit(X_train, [y_train[:,0], y_train[:,1], y_train[:,2]],
          epochs=50,
          batch_size=32,
          validation_data=(X_test, [y_test[:,0], y_test[:,1], y_test[:,2]]))

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Download
with open('rep_detection_model.tflite', 'wb') as f:
    f.write(tflite_model)

!apt-get install xxd
!xxd -i rep_detection_model.tflite > rep_detection_model.h
files.download('rep_detection_model.h')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/BarbellAI/deadlift_*.csv'